In [ ]:
%pip install langchain
%pip install openai

In [ ]:
import os, yaml

In [ ]:
# Add your Spotify and OpenAI API keys here
%env SPOTIPY_CLIENT_ID= 
%env SPOTIPY_CLIENT_SECRET=
%env SPOTIPY_REDIRECT_URI=
%env OPENAI_API_KEY =

In [ ]:
!wget https://raw.githubusercontent.com/openai/openai-openapi/master/openapi.yaml
!mv openapi.yaml openai_openapi.yaml
!wget https://raw.githubusercontent.com/APIs-guru/openapi-directory/main/APIs/spotify.com/1.0.0/openapi.yaml
!mv openapi.yaml spotify_openapi.yaml

In [ ]:
from langchain.agents.agent_toolkits.openapi.spec import reduce_openapi_spec

In [ ]:
with open("openai_openapi.yaml") as f:
    raw_openai_api_spec = yaml.load(f, Loader=yaml.Loader)
openai_api_spec = reduce_openapi_spec(raw_openai_api_spec)

with open("spotify_openapi.yaml") as f:
    raw_spotify_api_spec = yaml.load(f, Loader=yaml.Loader)
spotify_api_spec = reduce_openapi_spec(raw_spotify_api_spec)

In [ ]:
import spotipy.util as util
from langchain.requests import RequestsWrapper

def construct_spotify_auth_headers(raw_spec: dict):
    scopes = list(raw_spec['components']['securitySchemes']['oauth_2_0']['flows']['authorizationCode']['scopes'].keys())
    access_token = util.prompt_for_user_token(scope=','.join(scopes))
    return {
        'Authorization': f'Bearer {access_token}'
    }

headers = construct_spotify_auth_headers(raw_spotify_api_spec)
requests_wrapper = RequestsWrapper(headers=headers)

In [ ]:
from langchain.llms.openai import OpenAI
from langchain.agents.agent_toolkits.openapi import planner
llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

In [ ]:
spotify_agent = planner.create_openapi_agent(spotify_api_spec, requests_wrapper, llm)
user_query = " " #Enter your prompt here
spotify_agent.run(user_query)